In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np
import statsmodels.api as sm
from scipy import stats
import os
from scipy import stats

from mri_data import file_manager as fm

In [ ]:
drive_root = fm.get_drive_root()
curr_dir = Path(os.getcwd())
data_dir = curr_dir / "data0"

subject_sessions = pd.read_csv("/home/srs-9/Projects/ms_mri/analysis/thalamus/data0/subject-sessions.csv", 
    index_col="sub")

cp_vols = pd.read_csv(data_dir / "choroid_aschoplex_volumes.csv", index_col="subid")
tiv_data = pd.read_csv(data_dir / "tiv_data.csv", index_col="subid")
clinical_data = pd.read_csv(data_dir / "clinical_data_processed.csv", index_col="subid")
df = cp_vols.join([tiv_data, clinical_data])

df_z = df.copy()
numeric_vars = [col for col in df_z.columns if pd.api.types.is_numeric_dtype(df_z[col])]
df_z[numeric_vars] = df_z[numeric_vars].apply(stats.zscore, nan_policy="omit")

In [ ]:
model_data = df_z.copy()
model_data = model_data[model_data['dz_type2'].isin(["MS", "!MS"])]

model_data = pd.concat((model_data, pd.get_dummies(model_data['dz_type2'], dtype="int")), axis=1)

formula = "dz_type2 ~ choroid_volume + age + Female + tiv"
res = sm.Logit.from_formula(formula, data=model_data).fit()
print(res.summary())

ValueError: endog has evaluated to an array with multiple columns that has shape (486, 2). This occurs when the variable converted to endog is non-numeric (e.g., bool or str).